In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import os
import cv2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
import base64

# --- Constantes de configuration ---
EFFICIENTNET_INPUT_SIZE = (224, 224)
PHOTOS_DIR_PATH = 'data/photos/'
# --- Chargement des modèles et des données ---
@st.cache_resource
def charger_modeles():
    """Charge les modèles et les objets nécessaires."""
    try:
        feature_extractor = EfficientNetB0(include_top=False, weights='imagenet', pooling='avg')
        scaler = joblib.load('scaler.joblib')
        pca = joblib.load('pca.joblib')
        model = joblib.load('random_forest_model.joblib')
        label_encoder = joblib.load('label_encoder.joblib')
        
        # Charger les données d'entraînement pour l'analyse des mots-clés
        entrainement_data = joblib.load("entrainement.joblib")
        X_train_text = entrainement_data['X_train']
        y_train = entrainement_data['y_train']
        
        return feature_extractor, scaler, pca, model, label_encoder, X_train_text, y_train
    except Exception as e:
        st.error(f"Erreur lors du chargement des modèles/données : {e}")
        return None, None, None, None, None, None, None

feature_extractor, scaler, pca, model, label_encoder, X_train_text, y_train = charger_modeles()

def preprocess_image(image):
    """Prétraite une image pour EfficientNetB0."""
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, EFFICIENTNET_INPUT_SIZE)
    img_expanded = np.expand_dims(img_resized, axis=0)
    img_preprocessed = preprocess_input(img_expanded)
    return img_preprocessed

def extract_image_features(image, model):
    """Extrait les caractéristiques d'une image."""
    img_preprocessed = preprocess_image(image)
    return model.predict(img_preprocessed, verbose=0)[0]

def predict_topic(image, model_obj, scaler_obj, pca_obj):
    """Prédit le topic d'une image ou d'un texte."""
    if isinstance(image, np.ndarray):  # Si l'entrée est une image
        img_features = extract_image_features(image, feature_extractor)
        img_features_scaled = scaler_obj.transform(img_features.reshape(1, -1))
        if pca_obj is not None:
            img_features_pca = pca_obj.transform(img_features_scaled)
        else:
            img_features_pca = img_features_scaled
        prediction = model_obj.predict(img_features_pca)[0]
    elif isinstance(image, str):  # Si l'entrée est du texte
        text_features = scaler_obj.transform(image) # scaler est déjà fit sur les features efficientnet
        if pca_obj is not None:
            text_features_pca = pca_obj.transform(text_features)
        else:
            text_features_pca = text_features
        prediction = model_obj.predict(text_features_pca)[0]
    else:
        raise ValueError("Entrée non valide. Doit être une image (np.ndarray) ou du texte (str).")
    return label_encoder.inverse_transform([prediction])[0]  # Décode la prédiction

def get_tsne_visualization(features, labels, new_text_feature=None, new_text_label=None):
    """
    Calcule et affiche la visualisation t-SNE des caractéristiques,
    en incluant éventuellement un nouveau point de texte.
    """
    all_features = np.copy(features)  # Copie pour ne pas modifier l'original
    all_labels = list(labels)

    if new_text_feature is not None and new_text_label is not None:
        all_features = np.vstack([all_features, new_text_feature])
        all_labels.append(new_text_label)

    # Vérification que all_features a au moins 2 éléments
    if len(all_features) < 2:
        st.warning("Pas assez de données pour appliquer t-SNE.")
        return None
        
    perplexity_value = min(30, len(all_features) - 1)
    tsne = manifold.TSNE(n_components=2, init='pca', random_state=42, perplexity=perplexity_value)
    X_tsne = tsne.fit_transform(all_features)

    plt.figure(figsize=(10, 8))
    
    # Créer un mappage de couleurs basé sur les étiquettes
    unique_labels = list(set(all_labels))
    palette = sns.color_palette("viridis", n_colors=len(unique_labels))
    label_color_map = dict(zip(unique_labels, palette))

    # Assigner des couleurs aux points
    colors = [label_color_map[label] for label in all_labels]
    
    sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=all_labels, palette=label_color_map) # Utilisez le mappage ici
    
    plt.title('Visualisation t-SNE des topics, incluant le nouveau texte')
    plt.xlabel('t-SNE Composante 1')
    plt.ylabel('t-SNE Composante 2')
    plt.legend(title="Topic")

    # Convertir le graphique en image pour Streamlit
    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image_png = buf.getvalue()
    buf.close()
    plt.close()
    
    return image_png
    
def get_top_keywords(corpus, labels, n_keywords=10):
    """
    Extrait les mots-clés les plus importants pour chaque topic à partir du corpus d'entraînement.

    Args:
        corpus (pd.Series): Le corpus de texte d'entraînement.
        labels (pd.Series): Les étiquettes (topics) correspondant au corpus.
        n_keywords (int): Le nombre de mots-clés à extraire par topic.

    Returns:
        dict: Un dictionnaire où les clés sont les topics et les valeurs sont des listes de mots-clés.
    """
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()
    
    keywords_by_topic = {}
    for topic in labels.unique():
        # Sélectionne les documents appartenant à ce topic
        topic_docs_indices = np.where(labels == topic)[0]
        topic_tfidf_matrix = tfidf_matrix[topic_docs_indices]
        
        # Calcule la somme des scores TF-IDF pour chaque mot dans le topic
        sum_tfidf_scores = topic_tfidf_matrix.sum(axis=0).A1  # Convertit en array 1D
        
        # Obtient les indices des mots avec les scores TF-IDF les plus élevés
        top_keyword_indices = sum_tfidf_scores.argsort()[-n_keywords:][::-1]
        
        # Obtient les mots-clés correspondants
        top_keywords = [feature_names[index] for index in top_keyword_indices]
        keywords_by_topic[topic] = top_keywords
    return keywords_by_topic

def summarize_keywords(keywords_dict, n_top=3):
    """
    Génère une description concise des mots-clés pour chaque topic.

    Args:
        keywords_dict (dict): Dictionnaire de mots-clés par topic.
        n_top (int): Nombre de mots-clés principaux à utiliser dans le résumé.

    Returns:
        dict: Dictionnaire avec les résumés pour chaque topic.
    """
    summaries = {}
    for topic, keywords in keywords_dict.items():
        if len(keywords) > n_top:
            top_keywords = keywords[:n_top]
            summaries[topic] = f"Principaux thèmes : {', '.join(top_keywords)}"
        elif len(keywords) > 0:
            summaries[topic] = f"Principaux thèmes : {', '.join(keywords)}"
        else:
            summaries[topic] = "Aucun mot-clé pertinent trouvé"
    return summaries

def main():
    """Fonction principale de l'application Streamlit."""
    st.title("Classification de Topics d'Images et de Textes")
    st.write("Ce modèle classe des images et des textes dans différents topics.")

    # Sélecteur pour choisir le type d'entrée
    input_type = st.radio("Type d'entrée :", ["Image", "Texte"])

    if input_type == "Image":
        uploaded_file = st.file_uploader("Télécharger une image", type=["jpg", "png", "jpeg"])
        if uploaded_file is not None:
            # Lire l'image téléchargée
            image = cv2.imdecode(np.frombuffer(uploaded_file.read(), np.uint8), cv2.IMREAD_COLOR)
            st.image(image, caption="Image téléchargée", use_column_width=True)
            
            if st.button("Prédire le topic de l'image"):
                try:
                    predicted_topic = predict_topic(image, model, scaler, pca)
                    st.success(f"Topic prédit : {predicted_topic}")
                except Exception as e:
                    st.error(f"Erreur lors de la prédiction : {e}")
                    
    elif input_type == "Texte":
        new_text = st.text_area("Entrer un texte pour la classification :", "C'est un document intéressant.")
        if st.button("Prédire le topic du texte"):
            try:
                # Prédiction du topic du nouveau texte
                new_text_features = scaler.transform(feature_extractor.predict(preprocess_input(np.expand_dims(np.zeros(EFFICIENTNET_INPUT_SIZE, dtype=np.uint8), axis=0)), verbose=0)[0].reshape(1,-1)) # vecteur nul
                predicted_topic_text = predict_topic(new_text_features, model, scaler, pca)
                st.success(f"Topic prédit pour le texte : {predicted_topic_text}")
                
                # Visualisation t-SNE avec le nouveau point de texte
                tsne_image = get_tsne_visualization(np.stack(processed_features_X), processed_labels_y, new_text_features, predicted_topic_text)
                if tsne_image:
                    st.subheader("Visualisation t-SNE avec le texte ajouté")
                    st.image(tsne_image, caption="Visualisation t-SNE", use_column_width=True)
                else:
                    st.warning("La visualisation t-SNE n'a pas pu être générée.")

            except Exception as e:
                st.error(f"Erreur lors de la prédiction : {e}")

    # Analyse des mots-clés par topic
    st.header("Analyse des mots-clés par topic")
    try:
        keywords_by_topic = get_top_keywords(X_train_text, y_train)
        summaries = summarize_keywords(keywords_by_topic)
        for topic, summary in summaries.items():
            st.subheader(f"Topic : {topic}")
            st.write(summary)
    except Exception as e:
        st.error(f"Erreur lors de l'analyse des mots-clés : {e}")

if __name__ == "__main__":
    main()
